In [1]:
import sys
import os

__file__ = 'helper_funcs.py'
file_path = os.path.abspath(os.path.join(os.path.dirname( __file__ ), '..'))
sys.path.append(file_path)

import helper_funcs as hf


In [51]:
raw_input = '''...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....'''

mod_input = hf.modify_raw(raw_input)

def find_expanse(input:list)->list:
    #horizontal
    vert = []
    hor = []
    for n,i in enumerate(input): 
        if all(v=='.' for v in i):
            hor.append(n)
    #vertical
    for i in range(len(input[0])):
        tmp = []
        for j in input:
            tmp.append(j[i])
        
        if all(q=='.' for q in tmp):
            vert.append(i)

    return [hor,vert]


def adjust_map(input:list)->list:
    expanses = find_expanse(mod_input)
    copy = mod_input.copy()
    #horizontal
    gen_template = ''.join(['.' for i in range(len(input[0]))])
    t = 0
    for i in expanses[0]:
        copy.insert(i+t,gen_template)
        t+=1
    
    #vert
    t = 0
    for j in expanses[1]: #2 5 8
        for z in range(len(copy)):  #line by line number
            tmp = list(copy[z])
            tmp.insert(j+t,'.')
            copy[z] = ''.join(tmp)
        t+=1

    return copy


def find_galaxies(map:list)->list:
    galaxy_coors = []
    for i in range(len(map[0])):
        for j in range(len(map)):
            coor = (i,j)
            if map[j][i] == '#':
                galaxy_coors.append(coor)
    return galaxy_coors


new_map = adjust_map(mod_input)
find_galaxies(new_map)

[(0, 2), (0, 11), (1, 6), (4, 0), (5, 11), (8, 5), (9, 1), (9, 10), (12, 7)]

In [67]:
def distance(c1:tuple,c2:tuple):
    tot = 0
    if len(c1) == len(c2):
        for i in range(len(c1)):
            tot+= (c2[i]-c1[i])**2
    return tot**(1/2)

def ceil(a):
    return a//1 + bool(a%1)

ceil(distance((1,6),(5,11)))

7.0

In [137]:
def get_dim(matrix:list) -> tuple:
    #returns the dimensions of a matrix
    return (len(matrix[0]),len(matrix))

def clamp(matrix:list,coord:tuple)->bool:
    #mchecks the coordinate to make sure it's within the dim of the matrix, returns false of it's outside
    dim = get_dim(matrix)
    if (coord[0]< 0) or (coord[0]>dim[0]-1) or (coord[1]<0) or (coord[1]>dim[1]-1):
        return False
    else:
        return True

def neighbors(coor:tuple) -> list:
    #finds the neighbors of a cell including diagonal
    coords = []
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            if (abs(i)==abs(j)): #omit corners, make sure it's only 1 moved and the other constant
                pass
            else:
                coords.append((coor[0]+i,coor[1]+j))
    return coords

def find_cell(matrix:list,coor:tuple) -> str:
    #finds the value of a cell
    x = coor[0]
    y = coor[1]
    return matrix[y][x]

In [120]:
neighbors((1,1))

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]

In [141]:
#a* implementation

def astar(start:tuple,end:tuple):
    graph = []
    for j in range(0,max(start[1],end[1])+1):
        leaf = []
        for i in range(0,max(start[0],end[0])+1):
            h = distance((i,j),end) #heuristic
            
            leaf.append(h)
        graph.append(leaf)

    min_energy = find_cell(graph,start)
    next_cell = start
    l = 0
    tmp_map = new_map.copy()  ##uh oh global variable wee woo
    while min_energy != 0:
        for i in neighbors(next_cell):
            if clamp(graph,i):
                cell_energy = find_cell(graph,i)
                if cell_energy < min_energy:
                    min_energy = cell_energy
                    next_cell = i
       
        x = next_cell[0]
        y = next_cell[1]
        temp_list = list(tmp_map[y])
        temp_list[x] = 'x'
        tmp_map[y] = ''.join(temp_list)
        
     
        l +=1
    # print('\n'.join(tmp_map))

    return l

###########

new_map = adjust_map(mod_input)
galaxies = find_galaxies(new_map)

lengths = {}
for i in galaxies:
    for j in galaxies:
        if i != j:
            v = astar(i,j)
            lengths.setdefault(''.join(sorted([str(i),str(j)])),v)

sum(lengths.values())


374